# Exploration des techniques de clustering

Le but de ce tp est de faire face Ã  la problÃ©matique: 
<center style="color:red" >  Voici XXX documents -bruts, non Ã©tiquetÃ©s-... Comment les valoriser? Les exploiter? Les comprendre? Les rÃ©sumer? </center>

Nous avons vu dans les sÃ©ances prÃ©cÃ©dentes comment reprÃ©senter les donnÃ©es textuelles sous forme de sacs de mots:
$$X = 
	\begin{matrix} 
	 & \textbf{t}_j \\
	 & \downarrow \\
	\textbf{d}_i \rightarrow &
	\begin{pmatrix} 
	x_{1,1} & \dots & x_{1,D} \\
	\vdots & \ddots & \vdots \\
	x_{N,1} & \dots & x_{N,D} \\
	\end{pmatrix}
	\end{matrix}
	$$

A partir de cette reprÃ©sentation, les questions qui se posent sont les suivantes:
1. Quel algorithme de clustiering choisir?
    - K-means, LSA, pLSA, LDA
1. Quels rÃ©sultats attendre?
    - qualitÃ©, bruit, exploitabilitÃ© immÃ©diate etc...
1. Quelles analyses qualitatives effectuer pour comprendre les groupes?
1. Comment boucler, itÃ©rer pour amÃ©liorer la qualitÃ© du processus?



In [2]:
import numpy as np
import matplotlib.pyplot as plt

import codecs
import re
import os.path
import sklearn

## Chargement des donnÃ©es



In [3]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

In [28]:
# conversion BoW + tf-idf

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer() # TfidfVectorizer(max_features=500)
vectors = vectorizer.fit_transform(newsgroups_train.data)
print(vectors.shape)

# mesure de la sparsitÃ© = 157 mots actif par document sur 130000 !!
print(vectors.nnz / float(vectors.shape[0]))

(11314, 500)
70.5516174650875


In [29]:
# retrouver les mots
print([(i,vectorizer.get_feature_names()[i]) \
       for i in np.random.randint(vectors.shape[1], size=10)])

[(54, 'anything'), (269, 'maybe'), (100, 'case'), (132, 'doesn'), (152, 'everything'), (46, 'american'), (287, 'nasa'), (444, 'type'), (66, 'available'), (384, 'show')]


In [30]:
# gestion des Ã©tiquettes (pour l'Ã©valuation seulemnet en non-supervisÃ©)
Y = newsgroups_train.target
print(Y[:10]) # numÃ©rique
print([newsgroups_train.target_names[i] for i in Y[:10]]) # vraie classe

[ 7  4  4  1 14 16 13  3  2  4]
['rec.autos', 'comp.sys.mac.hardware', 'comp.sys.mac.hardware', 'comp.graphics', 'sci.space', 'talk.politics.guns', 'sci.med', 'comp.sys.ibm.pc.hardware', 'comp.os.ms-windows.misc', 'comp.sys.mac.hardware']


# Tests prÃ©liminaires

CommenÃ§ons par le commencement: tout problÃ¨me non-supervisÃ© (ou presque) doit Ãªtre analysÃ© en premier lieu avec les $k$-means!


In [31]:
from sklearn.cluster import KMeans
# Algo => risque de prendre du temps si le vocabulaire n'a pas Ã©tÃ© rÃ©duit !!
# Note: on dirait que l'algo transforme les donnÃ©es en dense vector=> catastrophe pour nous !!!
# => limitation du nombre d'itÃ©ration arbitraire + limitation du vocabulaire

kmeans = KMeans(n_clusters=20, random_state=0, max_iter=10).fit(vectors)

In [ ]:
# analyse

# recupÃ©ration des proto:
kmeans.cluster_centers_

# mots les plus importants par cluster => TODO
# version print / version word cloud

### Limites

- Limites liÃ©es Ã  la description
    - trop de mots
    - trop de mots frÃ©quents qui dÃ©routent l'algorithme
    - ...
- Limites liÃ©es Ã  l'algorithme
    - distance euclidienne absurde

Les limites algorithmiques vont Ãªtre rÃ©solues en changeant d'algorithme... Les limites de reprÃ©sentation des donnÃ©es seront rÃ©solues par votre capacitÃ© en ingÃ©nierie.


Algorithmes Ã  tester:
- LSA
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD
- LDA
https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

**Note:** pour des tests rapides, il est plus simple de rester dans le cadre de scikit-learn... NÃ©anmoins, dans un milieu industriel, il faudrait exploiter des outils plus efficaces comme ceux prÃ©sents dans la librairie ```gensim```. Si vous vous sentez Ã  l'aise avec la donnÃ©e textuelles, allez directement vers ces outils:
https://radimrehurek.com/gensim/models/ldamodel.html

## Evaluation des performances

Les performances sont trÃ¨s dures Ã  Ã©valuer en clustering... Ce qui explique que cette Ã©valuation est souvent, au moins partiellement, qualitative (=Ã©tudiÃ©e Ã  l'oeil, sur des exemples ou des paramÃ¨tres). 
Afin d'Ã©viter de faire n'importe quoi, il faut aussi rÃ©flÃ©chir Ã  des mÃ©triques quantitatives.

### Qualitatif

Analyser le vocabulaire des diffÃ©rents clusters
1. En terme de mots les plus frÃ©quents, les plus probables ou de dimensions des vecteurs propres les plus fortes.
1. En terme de mots discriminants
    - construction de critÃ¨re de contraste (type odd's ratio) entre la prÃ©sence dans une classe et prÃ©sence dans les autres classe
1. Affichage 
    - des 10 ou 20 mots critiques de chaque classe ```print```
    - sous la forme de word cloud
    - affichage interactif avancÃ©: http://www.kennyshirley.com/LDAvis/
        - pour une version intÃ©grable dans un notebook: https://github.com/bmabey/pyLDAvis
        - merci de l'utiliser **aprÃ¨s avoir compris le principe de rÃ©duction de la dimensionalitÃ© pour les clusters**
    
### Quantitatif

Pour donner des chiffres, il faut des Ã©tiquettes. C'est rarement le cas sur des jeux de donnÃ©es industriels... Mais c'est bon dans un cadre acadÃ©mique comme 20 newsgroups!

**ProblÃ¨me:** Comme nos algorithmes sont non supervisÃ©s, les sorties (bien que catÃ©gorielles) ne sont pas alignÃ©es avec l'encodage des Ã©tiquettes du jeu de donnÃ©es. Il faut trouver des astuces.

1. Etude basique sur la taille des clusters
    - est ce qu'une classe n'a pas tout attrapÃ©?
1. PuretÃ© = extraction de la classe majoritaire dans un cluster + calcul de la puretÃ© du cluster
    - 1 score par cluster par dÃ©faut
    - agrÃ©gation par somme pondÃ©rÃ©e sur la taille des clusters
1. Indice de Rand  https://fr.wikipedia.org/wiki/Indice_de_Rand
1. MÃ©trique adaptÃ©e Ã  une hypothÃ¨se spÃ©cifique


## Vers une version plus Ã©voluÃ©e des algorithmes

1. Si l'un des clusters attirÃ© toutes les donnÃ©es: Ãªtes-vous capable de supprimer ce cluster et de simplement rÃ©partir les Ã©chantillons dans les autres catÃ©gories?

1. Si vous avez une idÃ©e vague des thÃ©matiques que vous souhaitez voir isolÃ©es... 
    - trouver 10 mots dans chaque catÃ©gories
    - biaiser l'initialisation pour attirer ces classes

1. Si vous mettez un utilisateur dans la boucle
    - passer en mode supervisÃ© multiclasse et exploiter les feedbacks de l'utilisateur pour forcer le passage d'un document dans la classe d'Ã  cotÃ© 
        - Naive Bayes, SVM ou autre...
    - rÃ©flÃ©chir Ã  une approche active qui sÃ©lectionne les documents les plus intÃ©ressants Ã  montrer Ã  l'utilisateur pour lui demander un Ã©tiquetage.